In [1]:
%load_ext autoreload
%autoreload 2
%cd /home/ftei-dsw/projects/weather-gan

/home/ftei-dsw/projects/weather-gan


In [12]:
from src.model import Generator, Discriminator
from src.trainer import WeatherGenerator
from src.data import DataModule
from pytorch_lightning.trainer import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from src.utils import get_version

import torch

# start training

In [28]:
from typing import List
from dataclasses import dataclass, field

@dataclass
class Config:
    
    scratch_path = '/home/ftei-dsw/data/weather-gan'
    tensorboard_path: str = f'{scratch_path}/tensorboard/'
    checkpoint_path: str = f'{scratch_path}/checkpoints/'
    config_path: str = f'{scratch_path}/config-files/'
    results_path: str = f'{scratch_path}/results/'
    input_fname: str = f'{scratch_path}/datasets/monthly_gfdl_historical.nc'
    target_fname: str = f'{scratch_path}/datasets/daily_gfdl_historical.nc'

    train_start: int = 1850
    train_end: int = 1986
    valid_start: int = 2000
    valid_end: int = 2014
    test_start: int = 2000
    test_end: int = 2014
    
    model_name: str = 'weather-gan'
    
    num_variables: int = 2
    epochs: int = 10
    train_batch_size: int = 32
    test_batch_size: int = 64
    lr: float = 0.0002
    beta1: float = 0.5
    beta2: float = 0.999
    
    latent_dim: int = 8
    apply_dropout: bool = False
    generator_channels: int = 64
    generator_num_resblocks: int = 5
    generator_num_downsampling: int = 2

    discriminator_channels: int = 32
    discriminator_num_layers = 3
    n_critic = 5


In [ ]:
def training(config):

    tb_logger = TensorBoardLogger(config.tensorboard_path,
                                  name=config.model_name,
                                  default_hp_metric=False,
                                  version = get_version())
    
    model = WeatherGenerator(config)
                             
    data = DataModule(config)
    data.setup('fit')
    
    trainer = Trainer(gpus=1,
                     logger=tb_logger)
    
    trainer.fit(model, data)

training(Config())

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type          | Params
------------------------------------------------
0 | generator     | Generator     | 6.7 M 
1 | discriminator | Discriminator | 693 K 
------------------------------------------------
7.4 M     Trainable params
0         Non-trainable params
7.4 M     Total params
29.480    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

In [13]:
a = torch.ones(1,22,3,4)
b = torch.ones(1,22,3,4)
torch.cat([a,b], dim=1).shape

torch.Size([1, 44, 3, 4])

In [17]:
a = torch.ones(3,4,4)
import torchvision
torchvision.utils.make_grid(a)

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])

In [3]:
#from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint, progress
#    from pytorch_lightning.loggers import TensorBoardLogger
#
#def get_cycle_gan_callbacks(checkpoint_path, continued_training=False) -> list:
#
#    lr_logger = LearningRateMonitor(logging_interval='epoch')
#
#    checkpoint_callback = ModelCheckpoint(monitor="g_tot_val_loss",
#                                          dirpath=checkpoint_path,
#                                          save_top_k = 50,
#                                          #period=2,
#                                          save_last=True)
#    if continued_training:
#        checkpoint_callback.CHECKPOINT_NAME_LAST = "{epoch}-last-continued"
#
#    #progress_bar = progress.TQDMProgressBar(refresh_rate=2)
#
#    #callbacks = [lr_logger, checkpoint_callback, progress_bar]
#    callbacks = [lr_logger, checkpoint_callback]
#
#    return callbacks